In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df0 = pd.read_excel('/content/drive/MyDrive/PTDL/Customer Segment Analysis/dataset/Orders.xlsx')
df0

TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
df0.isna().sum()

In [ ]:
df0.describe()


In [ ]:
#------------------------------------------------------------Missing------------------------------------------------------------------------------------------
#Điền CustomerID cho những đơn hàng bị thiếu nếu đơn hàng đó tồn tại InvoiceNo
#Sau đó xóa tất cả những đơn bị thiếu CustomerID hoặc InvoiceNo, tạo df mới k có NaN
InvoiceCustomer_dict = df0.groupby('InvoiceNo')['CustomerID'].first().to_dict()
InvoiceCustomer_dict


In [ ]:
df0['CustomerID'] = df0.apply(lambda row: row['CustomerID'] if pd.notna(row['CustomerID'])
                    else InvoiceCustomer_dict.get(row['InvoiceNo'], row['CustomerID']), axis=1)
df = df0.dropna(subset=['CustomerID', 'InvoiceNo'],how = 'any')

df.describe()

In [ ]:


#-----------------------------------------------------------Outliners------------------------------------------------------------------------------------------
#Xóa những dòng có UnitPrice âm
df = df[df['UnitPrice'] > 0]
#Xóa giá trị trùng
df.drop_duplicates(inplace = True)
df.describe()

Phân tích dữ liệu

In [ ]:
#Tính các chỉ số RFM
#Recency
import datetime
cur_date = max(df['InvoiceDate'] + datetime.timedelta(days= 1))
df['InvoiceDate']= pd.to_datetime(df['InvoiceDate'])
Recency = df.groupby('CustomerID').agg({'InvoiceDate': lambda d:  (cur_date -d.max()).days  })



#Frequency
Frequency = df.groupby('CustomerID')['InvoiceNo'].count()

#Montary Value
df['Total'] = df['UnitPrice'] * df['Quantity']
Montary = df.groupby('CustomerID')['Total'].sum()

#RFM
rfm = pd.concat([Recency,Frequency,Montary],axis = 1)
rfm.columns = (['Recency','Frequency','Montary Value'])
rfm.describe()

In [ ]:
plt.figure(figsize=(12, 6))
n, bins, patches = plt.hist(rfm['Recency'], bins=33, alpha=0.9)
plt.title('Distribution of Recency')
plt.xlabel('Recency')
plt.ylabel('Number of Customers')

for i in range(len(patches)):
    plt.text(patches[i].get_x() + patches[i].get_width()/2, n[i], int(n[i]), ha='center', va='bottom')

plt.show()

In [ ]:
df_subset_f = rfm[rfm['Frequency'] > 1000]
plt.figure(figsize=(12, 6))
n, bins, patches = plt.hist(rfm['Frequency'][rfm['Frequency'] < 1100], bins=35, alpha=0.9, label='Frequency < 1000')
plt.hist(df_subset_f['Frequency'], bins=bins, alpha=0.5, label='Frequency > 1000')
plt.title('Distribution of Frequency')
plt.xlabel('Frequency')
plt.ylabel('Number of Customers')
plt.legend()
for i in range(len(patches)):
    if i < len(patches) - 1:
        plt.text(patches[i].get_x() + patches[i].get_width() / 2, n[i], int(n[i]), ha='center', va='bottom')
    else:
        plt.text(patches[i].get_x() + patches[i].get_width() -15, n[i] - 83, f"{int(n[i])}\n>1000", ha='center', va='bottom')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
n, bins, patches = plt.hist(rfm['Montary Value'][rfm['Montary Value']< 50000 ], bins=50, alpha=1-0.1)
plt.yscale('log')
plt.title('Monetary Value (log scale)')
plt.xlabel('Monetary Value')
plt.ylabel('Number of Customers (log scale)')
plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
for i in range(len(patches)):
    if n[i] > 0:
        plt.text(patches[i].get_x() + patches[i].get_width()/2, n[i], int(n[i]), ha='center', va='bottom')
plt.show()

In [ ]:
#Rời rạc hóa các chỉ số RFM
bins_r = [0, 30, 60, 180, float('inf')]
labels_r = [4, 3, 2, 1]

bins_f = [0, 25, 50, 100, float('inf')]
labels_f = [1, 2, 3, 4]

bins_m = [-4287, 300, 800, 1700, float('inf')]
labels_m = [1, 2, 3, 4]

# Rời rạc hóa các cột theo khoảng và nhãn đã xác định

r = pd.cut(rfm['Recency'], bins = bins_r, labels=labels_r)
f = pd.cut(rfm['Frequency'],bins=bins_f, labels=labels_f)
m = pd.cut(rfm['Montary Value'],bins=bins_m, labels=labels_m )
rfm['R-F-M'] = r.astype(str)+ '-' + f.astype(str) + '-' +m.astype(str)
rfm


In [ ]:

Loyal_Customer = rfm[rfm['R-F-M'].isin(['4-4-4', '4-4-3', '4-3-4', '3-4-4', '3-3-4'])]



Potential_Loyalist =  rfm[rfm['R-F-M'].isin([  '4-4-2', '4-4-1', '4-3-3', '4-3-2', '4-3-1' ,'3-4-3', '3-4-2', '3-4-1', '3-3-3', '3-3-2', '3-3-1'])]
Recent_Customer =   rfm[rfm['R-F-M'].isin([  '4-2-2', '4-2-1', '4-1-2', '4-1-1', '3-2-2', '3-2-1', '3-1-2', '3-1-1'])]
Promising =  rfm[rfm['R-F-M'].isin([   '4-2-4', '4-2-3', '4-1-4', '4-1-3', '3-1-4', '3-1-3'])]
Customer_Needing_Attention = rfm[rfm['R-F-M'].isin([   '3-2-4', '3-2-3'])]
At_Risk = rfm[rfm['R-F-M'].isin([    '2-4-2', '2-4-1', '2-3-2', '2-3-1', '2-2-2', '1-4-2', '1-4-1', '1-3-2'])]
Cant_Lose_Them = rfm[rfm['R-F-M'].isin([    '2-4-4', '2-4-3', '2-3-4', '2-3-3', '2-2-4', '2-2-3', '2-1-4', '2-1-3', '1-4-4', '1-4-3', '1-3-4', '1-3-3', '1-2-4', '1-2-3', '1-1-4', '1-1-3'])]
Hibernating = rfm[rfm['R-F-M'].isin([    '2-2-1', '2-1-2', '2-1-1', '1-3-1', '1-2-2'])]
Lost = rfm[rfm['R-F-M'].isin([  '1-2-1', '1-1-2', '1-1-1'])]
Promising


In [ ]:
# Tên từng phân khúc
seg = ['Loyal_Customer', 'Potential_Loyalist',  'Promising','Recent_Customer', 'Customer_Needing_Attention', 'At_Risk ', 'Cant_Lose_Them', 'Hibernating','Lost']

# Số khách hàng của phân khúc
seg_customers = [len(Loyal_Customer), len(Potential_Loyalist),  len(Promising),len(Recent_Customer), len(Customer_Needing_Attention), len(At_Risk), len(Cant_Lose_Them), len(Hibernating), len(Lost)]

total_customers = sum(seg_customers)

# Tính phần trăm khách hàng của mỗi nhóm so với tổng số khách hàng
seg_percents = [seg_customers[i] / total_customers * 100 for i in range(len(seg_customers))]

#Vẽ
plt.figure(figsize=(10, 10))
wedges, texts, autotexts = plt.pie(seg_percents, labels=None, autopct='%1.1f%%', startangle=140)


# Chú thích
note = []
for i in range(9):
  note.append( seg[i] + ': ' + str(seg_customers[i]))
plt.legend(wedges, note, title=('Total Customer: ' + str(total_customers)), loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
#Chia lại
loy_custome = len(Loyal_Customer)+ len(Potential_Loyalist) + len(Promising)
non_loy = total_customers - loy_custome
seg2 = ['Khách hàng thân thiết: ' +str(loy_custome), 'Khách hàng thông thường: '+ str(non_loy)]
sizes = [loy_custome, non_loy]
colors = ['#ffff14', '#069af3']


# Vẽ biểu đồ tròn
plt.figure(figsize=(8, 8))
explode = [0.05 if label == seg2[0] else -0.0001 for label in seg2]
plt.pie(sizes, labels=seg2, colors=colors, autopct='%1.0f%%',
       explode=explode ,startangle=320, labeldistance= 0.1)
plt.title('Tổng số khách hàng: ' + str(total_customers))








In [ ]:
#Đọc file CustomerDetails vào df_cu
df_cu = df0 = pd.read_excel('/content/drive/MyDrive/PTDL/Customer Segment Analysis/dataset/CustomerDetails.xlsx')
df_cu

In [ ]:
#Kết từng df phân khúc với bảng CustomerDetails qua thuộc tính CustomerID
df_loyal = pd.merge(df_cu, Loyal_Customer, on='CustomerID', how='inner')
[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1',
  'Phone 2','Email', 'Subscription Date','Website']]
df_loyal.to_excel('loyal.xlsx')
df_loyal


In [ ]:

df_potentail = pd.merge(df_cu, Potential_Loyalist, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_potentail.to_excel('potential.xlsx')
df_recent = pd.merge(df_cu, Recent_Customer, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_recent.to_excel('recent.xlsx')
df_prosiming = pd.merge(df_cu, Promising, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_prosiming.to_excel('promising.xlsx')
df_cu_need_att = pd.merge(df_cu, Customer_Needing_Attention, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_cu_need_att.to_excel('need_attention.xlsx')
df_at_risk = pd.merge(df_cu, At_Risk, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_at_risk.to_excel('at_risk.xlsx')
df_cant_lose = pd.merge(df_cu, Cant_Lose_Them, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_cant_lose.to_excel('cant_lose.xlsx')
df_hibernate = pd.merge(df_cu, Hibernating, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_hibernate.to_excel('hibernating.xlsx')
df_lost = pd.merge(df_cu, Lost, on='CustomerID', how='inner')[['CustomerID', 'First Name','Last Name', 'Company','City', 'Country','Phone 1', 'Phone 2','Email', 'Subscription Date','Website']]
df_lost.to_excel('lost.xlsx')